<a href="https://colab.research.google.com/github/MarianoChic09/MSc-ORT-Deep-Learning/blob/main/Clase%207/5_Bees_Template_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

%cd /content/drive/MyDrive/Colab Notebooks/Datasets/bees_dataset

## 1.1 Imports

In [ ]:
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout
import tensorflow as tf

import utils

## 1.2 Set random seeds

In [ ]:
np.random.seed(117)
tf.random.set_seed(117)

## 1.3 Global variables

In [ ]:
img_width = 100
img_height = 100
img_channels = 3

# 2. Carga de datos

In [ ]:
bees, bees_test_for_evaluation = utils.read_data()

In [ ]:
bees.head()

In [ ]:
bees_test_for_evaluation.head()

# 3. Análisis exploratorio de datos

## 3.1 Análisis descriptivo: Distribuciones, Scatterplots, Barplots...

In [ ]:
utils.value_counts(bees, 'subspecies')

In [ ]:
utils.value_counts(bees, 'location')

In [ ]:
utils.value_counts(bees, 'zip code')

Voy a usar zip code porque se repite Athens, Georgia en location y ya es un numero además.

In [ ]:
utils.value_counts(bees, 'caste')

Tambien voy a dropear caste porque es constante.

In [ ]:
utils.value_counts(bees, 'pollen_carrying')

In [ ]:
bees.dtypes

Voy a droppear health porque no esta en el test.

## 3.2 Ver imágenes

In [ ]:
utils.plot_images(bees, 'location', [0, 18, 24, 38, 45])

# 4. Clasificación

## 4.1. Data preprocessing
### 4.1.1 Particionamiento

In [ ]:
train_bees, val_bees, test_bees = utils.split(bees)

In [ ]:
train_bees


### 4.1.2 Carga de imágenes

In [ ]:
import os

os.chdir("/content/drive/MyDrive/Colab Notebooks/Datasets/bees_dataset")


In [ ]:
print(os.getcwd())


In [ ]:
print(os.listdir())
os.chdir("./data/imgs")
print(os.listdir())
os.chdir("../../")

In [ ]:
# os.chdir("../../")
# print(os.listdir())


In [ ]:
train_X, val_X, test_X, train_y, val_y, test_y = utils.load_images_and_target(train_bees,
                                                                              val_bees,
                                                                              test_bees,
                                                                              'subspecies',
                                                                              img_width,
                                                                              img_height,
                                                                              img_channels)

In [ ]:
optimizer = 'sgd'
loss = 'categorical_crossentropy'

In [ ]:
model1 = Sequential()
model1.add(Flatten(input_shape =(img_height, img_width, img_channels)))
model1.add(Dense(train_y.columns.size, activation = 'softmax'))
model1.compile(optimizer = optimizer, loss = loss, metrics = ['accuracy'])

## 4.3 Entrenamiento

### 4.2.2 Parámetros de transformación de imágenes (data augmentation)

In [ ]:
rotation_range = 15      # rotación aleatoria en grados entre 0 a rotation_range
zoom_range = 0.1         # zoom aleatorio
width_shift_range = 0.1  # desplazamiento horizontal aleatorio (fracción del total)
height_shift_range = 0.1 # desplazamiento vertical aleatorio (fracción del total)
horizontal_flip = True   # transposición horizontal
vertical_flip = True     # transposición horizontal

In [ ]:
batch_size = 10
epochs = 5
steps_per_epoch = 10
patience = 10
class_weights = utils.class_weights(bees, 'subspecies')

In [ ]:
class_weights

In [ ]:
training1, model1 = utils.train(model1,
                train_X,
                train_y,
                batch_size = batch_size,
                epochs = epochs,
                validation_data_X = val_X,
                validation_data_y = val_y,
                steps_per_epoch = steps_per_epoch,
                rotation_range = rotation_range,
                zoom_range = zoom_range,
                width_shift_range = width_shift_range,
                height_shift_range = height_shift_range,
                horizontal_flip = horizontal_flip,
                vertical_flip = vertical_flip,
                patience = patience,
                class_weights = class_weights
                               )

In [ ]:
def train(model,
                train_X,
                train_y,
                batch_size,
                epochs,
                validation_data_X,
				validation_data_y,
                steps_per_epoch,
                rotation_range,  # randomly rotate images in the range (degrees, 0 to rotation_range)
                zoom_range, # Randomly zoom image
                width_shift_range,  # randomly shift images horizontally (fraction of total width)
                height_shift_range,  # randomly shift images vertically (fraction of total height)
                horizontal_flip,  # randomly flip images
                vertical_flip,
				patience,
				class_weights):

	generator = ImageDataGenerator(
				featurewise_center=False,  # set input mean to 0 over the dataset
				samplewise_center=False,  # set each sample mean to 0
				featurewise_std_normalization=False,  # divide inputs by std of the dataset
				samplewise_std_normalization=False,  # divide each input by its std
				zca_whitening=False,  # apply ZCA whitening
				rotation_range=rotation_range,  # randomly rotate images in the range (degrees, 0 to rotation_range)
				zoom_range = zoom_range, # Randomly zoom image
				width_shift_range=width_shift_range,  # randomly shift images horizontally (fraction of total width)
				height_shift_range=height_shift_range,  # randomly shift images vertically (fraction of total height)
				horizontal_flip=horizontal_flip,  # randomly flip images
				vertical_flip=vertical_flip)


	generator.fit(train_X)
	#Train
	##Callbacks
	earlystopper = EarlyStopping(monitor='loss', patience=patience, verbose=1,restore_best_weights=True)



	training = model.fit(generator.flow(train_X,train_y, batch_size)
                        ,epochs=epochs
                        ,validation_data=(validation_data_X, validation_data_y)
                        ,steps_per_epoch=steps_per_epoch
                        ,callbacks=[earlystopper],
						class_weight = class_weights)

	return training, model


def combined_gen(image_gen, meta_data):
    for (x, y) in image_gen:
        yield [x, meta_data], y

def train_with_metadata(model,
                train_X,
                train_y,
                train_meta,
                batch_size,
                epochs,
                validation_data_X,
				validation_data_y,
                steps_per_epoch,
                rotation_range,  # randomly rotate images in the range (degrees, 0 to rotation_range)
                zoom_range, # Randomly zoom image
                width_shift_range,  # randomly shift images horizontally (fraction of total width)
                height_shift_range,  # randomly shift images vertically (fraction of total height)
                horizontal_flip,  # randomly flip images
                vertical_flip,
				patience,
				class_weights=None):

	datagen = ImageDataGenerator(
				featurewise_center=False,  # set input mean to 0 over the dataset
				samplewise_center=False,  # set each sample mean to 0
				featurewise_std_normalization=False,  # divide inputs by std of the dataset
				samplewise_std_normalization=False,  # divide each input by its std
				zca_whitening=False,  # apply ZCA whitening
				rotation_range=rotation_range,  # randomly rotate images in the range (degrees, 0 to rotation_range)
				zoom_range = zoom_range, # Randomly zoom image
				width_shift_range=width_shift_range,  # randomly shift images horizontally (fraction of total width)
				height_shift_range=height_shift_range,  # randomly shift images vertically (fraction of total height)
				horizontal_flip=horizontal_flip,  # randomly flip images
				vertical_flip=vertical_flip)

  train_gen = datagen.flow(train_X, train_y, batch_size=batch_size)
  combined_train_gen = combined_gen(train_gen, train_meta)

	#Train
	##Callbacks
	earlystopper = EarlyStopping(monitor='loss', patience=patience, verbose=1,restore_best_weights=True)



	training = model.fit(
        combined_train_gen,
        validation_data=([val_X, val_meta], val_y),
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        ,callbacks=[earlystopper],
        class_weight=class_weights,
        # ... other parameters
    )


  # model.fit(generator.flow(train_X,train_y, batch_size)
  #                       ,epochs=epochs
  #                       ,validation_data=(validation_data_X, validation_data_y)
  #                       ,steps_per_epoch=steps_per_epoch
  #                       ,callbacks=[earlystopper],
	# 					class_weight = class_weights)

	return training, model



def combined_gen(image_gen, meta_data):
    for (x, y) in image_gen:
        yield [x, meta_data], y

def train_with_metadata(model,
                train_X,
                train_y,
                train_meta,
                batch_size,
                epochs,
                validation_data_X,
				validation_data_y,
                steps_per_epoch,
                rotation_range,  # randomly rotate images in the range (degrees, 0 to rotation_range)
                zoom_range, # Randomly zoom image
                width_shift_range,  # randomly shift images horizontally (fraction of total width)
                height_shift_range,  # randomly shift images vertically (fraction of total height)
                horizontal_flip,  # randomly flip images
                vertical_flip,
				patience,
				class_weights=None):

	datagen = ImageDataGenerator(
				featurewise_center=False,  # set input mean to 0 over the dataset
				samplewise_center=False,  # set each sample mean to 0
				featurewise_std_normalization=False,  # divide inputs by std of the dataset
				samplewise_std_normalization=False,  # divide each input by its std
				zca_whitening=False,  # apply ZCA whitening
				rotation_range=rotation_range,  # randomly rotate images in the range (degrees, 0 to rotation_range)
				zoom_range = zoom_range, # Randomly zoom image
				width_shift_range=width_shift_range,  # randomly shift images horizontally (fraction of total width)
				height_shift_range=height_shift_range,  # randomly shift images vertically (fraction of total height)
				horizontal_flip=horizontal_flip,  # randomly flip images
				vertical_flip=vertical_flip)

  train_gen = datagen.flow(train_X, train_y, batch_size=batch_size)
  combined_train_gen = combined_gen(train_gen, train_meta)

	#Train
	##Callbacks
	earlystopper = EarlyStopping(monitor='loss', patience=patience, verbose=1,restore_best_weights=True)



	training = model.fit(
        combined_train_gen,
        validation_data=([val_X, val_meta], val_y),
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        ,callbacks=[earlystopper],
        class_weight=class_weights,
        # ... other parameters
    )


  # model.fit(generator.flow(train_X,train_y, batch_size)
  #                       ,epochs=epochs
  #                       ,validation_data=(validation_data_X, validation_data_y)
  #                       ,steps_per_epoch=steps_per_epoch
  #                       ,callbacks=[earlystopper],
	# 					class_weight = class_weights)

	return training, model

## 4.3 Evaluación del modelo

In [ ]:
utils.eval_model(training1, model1, test_X, test_y, 'subspecies')

## 4.4 Evaluación y generación de archivo para competencia Kaggle

In [ ]:
 df_subspecies = utils.load_test_and_generate_prediction_file(model1, img_width, img_height, img_channels)

In [ ]:
df_subspecies

# Transfer Learning

## Qué es Transfer Learning?

Transfer learning o aprendizaje por transferencia es un problema de investigación en el aprendizaje automático que se centra en almacenar el conocimiento adquirido mientras se resuelve un problema y se aplica a un problema diferente pero relacionado.

[Keras Models](https://keras.io/api/applications/)

In [ ]:
# example of loading the vgg16 model
from tensorflow.keras.applications.vgg16 import VGG16
# load model

model = VGG16(input_shape=(img_height, img_width, img_channels), include_top=False)
# model = VGG16(input_shape=(224, 224, 3), include_top=True)

# summarize the model
model.summary()

In [ ]:
for layer in model.layers[:-3]:
  layer.trainable = False
model.summary()

In [ ]:
from tensorflow.keras.models import Model

flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(7, activation='softmax')(class1)

# define new model
model = Model(inputs=model.inputs, outputs=output)

optimizer = 'sgd'
loss = 'categorical_crossentropy'

# compile the model
model.compile(optimizer = optimizer, loss = loss, metrics = ['accuracy'])

In [ ]:
model.summary()

## Entrenamiento

Parámetros de transformación de imágenes (data augmentation)

In [ ]:
rotation_range = 15      # rotación aleatoria en grados entre 0 a rotation_range
zoom_range = 0.1         # zoom aleatorio
width_shift_range = 0.1  # desplazamiento horizontal aleatorio (fracción del total)
height_shift_range = 0.1 # desplazamiento vertical aleatorio (fracción del total)
horizontal_flip = True   # transposición horizontal
vertical_flip = True     # transposición horizontal

In [ ]:
from sklearn.utils.class_weight import compute_sample_weight

def computing_class_weights(df, class_name):
    class_labels = {name: index for index, name in enumerate(np.unique(df[class_name]))}
    y_integers = np.array([class_labels[name] for name in df[class_name]])
    weights = compute_sample_weight(class_weight='balanced', y=y_integers)

    weight_dict = {}
    for class_index in class_labels.values():
        weight_dict[class_index] = weights[y_integers == class_index].mean()

    return weight_dict

import tensorflow as tf
from tensorflow.keras import backend as K

def weighted_accuracy(weight_dict):
    class_weights = tf.constant([weight_dict[i] for i in range(len(weight_dict))])

    def calc_weighted_accuracy(y_true, y_pred):
        y_true_labels = K.argmax(y_true, axis=1)
        y_pred_labels = K.argmax(y_pred, axis=1)

        correct_predictions = K.cast(K.equal(y_true_labels, y_pred_labels), dtype='float32')
        weights = K.gather(class_weights, y_true_labels)
        weighted_correct_predictions = correct_predictions * weights

        accuracy = K.sum(weighted_correct_predictions) / K.sum(weights)
        return accuracy

    return calc_weighted_accuracy


In [ ]:
batch_size = 10
epochs = 500
steps_per_epoch = 10
patience = 100

class_weights = computing_class_weights(bees, 'subspecies')


In [ ]:
class_weights

In [ ]:
training_vgg16, model = utils.train(model,
                train_X,
                train_y,
                batch_size = batch_size,
                epochs = epochs,
                validation_data_X = val_X,
                validation_data_y = val_y,
                steps_per_epoch = steps_per_epoch,
                rotation_range = rotation_range,
                zoom_range = zoom_range,
                width_shift_range = width_shift_range,
                height_shift_range = height_shift_range,
                horizontal_flip = horizontal_flip,
                vertical_flip = vertical_flip,
                patience = patience,
                class_weights = class_weights
                               )

In [ ]:
trainable_params = ((3*3*512)*1024+1024)+(1024*7+7)
trainable_params

## Evaluación del modelo

In [ ]:
utils.eval_model(training_vgg16, model, test_X, test_y, 'subspecies')

# Imbalanced Learning


# Archivo para Kaggle

In [ ]:
df_subspecies = utils.load_test_and_generate_prediction_file(model1, img_width, img_height, img_channels)
df_subspecies

In [ ]:
minority_images


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create a new ImageDataGenerator with the desired augmentations
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Assuming minority_images is a numpy array of images from the minority class
augmented_images = []
num_augmentations_per_image = 10  # You can adjust this value based on how many augmented images you want per original image

for img in minority_images:
    img = img.reshape((1,) + img.shape)  # Reshape the image
    i = 0
    for batch in datagen.flow(img, batch_size=1):
        augmented_images.append(batch[0])
        i += 1
        if i >= num_augmentations_per_image:
            break  # Avoid generator to loop indefinitely

# Now, `augmented_images` will contain the augmented images, and you can add these images to your training dataset.

# VGG 19

In [ ]:
# example of loading the vgg16 model
from tensorflow.keras.applications.vgg19 import VGG19
# load model

model = VGG19(input_shape=(img_height, img_width, img_channels), include_top=False)
# model = VGG16(input_shape=(224, 224, 3), include_top=True)

# summarize the model
model.summary()

In [ ]:
for layer in model.layers[:-3]:
  layer.trainable = False
model.summary()

In [ ]:
from tensorflow.keras.models import Model

flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(7, activation='softmax')(class1)

# define new model
model = Model(inputs=model.inputs, outputs=output)

optimizer = 'sgd'
loss = 'categorical_crossentropy'

# compile the model
model.compile(optimizer = optimizer, loss = loss, metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
rotation_range = 15      # rotación aleatoria en grados entre 0 a rotation_range
zoom_range = 0.1         # zoom aleatorio
width_shift_range = 0.1  # desplazamiento horizontal aleatorio (fracción del total)
height_shift_range = 0.1 # desplazamiento vertical aleatorio (fracción del total)
horizontal_flip = True   # transposición horizontal
vertical_flip = True     # transposición horizontal

In [ ]:
from sklearn.utils.class_weight import compute_sample_weight

def computing_class_weights(df, class_name):
    class_labels = {name: index for index, name in enumerate(np.unique(df[class_name]))}
    y_integers = np.array([class_labels[name] for name in df[class_name]])
    weights = compute_sample_weight(class_weight='balanced', y=y_integers)

    weight_dict = {}
    for class_index in class_labels.values():
        weight_dict[class_index] = weights[y_integers == class_index].mean()

    return weight_dict

In [ ]:
batch_size = 10
epochs = 500
steps_per_epoch = 10
patience = 100

class_weights = computing_class_weights(bees, 'subspecies')


In [ ]:
class_weights

In [ ]:
training_vgg19, model = utils.train(model,
                train_X,
                train_y,
                batch_size = batch_size,
                epochs = epochs,
                validation_data_X = val_X,
                validation_data_y = val_y,
                steps_per_epoch = steps_per_epoch,
                rotation_range = rotation_range,
                zoom_range = zoom_range,
                width_shift_range = width_shift_range,
                height_shift_range = height_shift_range,
                horizontal_flip = horizontal_flip,
                vertical_flip = vertical_flip,
                patience = patience,
                class_weights = class_weights
                               )

In [ ]:
utils.eval_model(training_vgg19, model, test_X, test_y, 'subspecies')

In [ ]:
df_subspecies = utils.load_test_and_generate_prediction_file(model1, img_width, img_height, img_channels)
df_subspecies

# ResNet 50

In [ ]:
from sklearn.utils.class_weight import compute_sample_weight

def computing_class_weights(df, class_name):
    class_labels = {name: index for index, name in enumerate(np.unique(df[class_name]))}
    y_integers = np.array([class_labels[name] for name in df[class_name]])
    weights = compute_sample_weight(class_weight='balanced', y=y_integers)

    weight_dict = {}
    for class_index in class_labels.values():
        weight_dict[class_index] = weights[y_integers == class_index].mean()

    return weight_dict

import tensorflow as tf
from tensorflow.keras import backend as K

def weighted_accuracy(weight_dict):
    class_weights = tf.constant([weight_dict[i] for i in range(len(weight_dict))])

    def calc_weighted_accuracy(y_true, y_pred):
        y_true_labels = K.argmax(y_true, axis=1)
        y_pred_labels = K.argmax(y_pred, axis=1)

        correct_predictions = K.cast(K.equal(y_true_labels, y_pred_labels), dtype='float32')
        weights = K.gather(class_weights, y_true_labels)
        weighted_correct_predictions = correct_predictions * weights

        accuracy = K.sum(weighted_correct_predictions) / K.sum(weights)
        return accuracy

    return calc_weighted_accuracy

In [ ]:
base_model = tf.keras.applications.ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:
# example of loading the vgg16 model
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
# load model

model = ResNet152V2(input_shape=(img_height, img_width, img_channels), include_top=False)
# model = VGG16(input_shape=(224, 224, 3), include_top=True)

# summarize the model
model.summary()

In [ ]:
for layer in model.layers[:-6]:
  layer.trainable = False
model.summary()

In [ ]:
from tensorflow.keras import Model, models, layers, optimizers

# flat1 = Flatten()(model.layers[-1].output)
# class1 = Dense(1024, activation='relu')(flat1)
# output = Dense(7, activation='softmax')(class1)

# # define new model
# model = Model(inputs=model.inputs, outputs=output)
# model.trainable = False

model = models.Sequential([
    model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),  # Optional: for regularization
    layers.Dense(7, activation='softmax')  # Adjust for the number of classes in your dataset
])

optimizer = optimizers.Adam(learning_rate=1e-4)
loss = 'categorical_crossentropy'

# compile the model
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
rotation_range = 180      # rotación aleatoria en grados entre 0 a rotation_range
zoom_range = 0.1         # zoom aleatorio
width_shift_range = 0.1  # desplazamiento horizontal aleatorio (fracción del total)
height_shift_range = 0.1 # desplazamiento vertical aleatorio (fracción del total)
horizontal_flip = True   # transposición horizontal
vertical_flip = True     # transposición horizontal

In [ ]:
from sklearn.utils.class_weight import compute_sample_weight

def computing_class_weights(df, class_name):
    class_labels = {name: index for index, name in enumerate(np.unique(df[class_name]))}
    y_integers = np.array([class_labels[name] for name in df[class_name]])
    weights = compute_sample_weight(class_weight='balanced', y=y_integers)

    weight_dict = {}
    for class_index in class_labels.values():
        weight_dict[class_index] = weights[y_integers == class_index].mean()

    return weight_dict

In [ ]:
batch_size = 10
epochs = 1500
steps_per_epoch = 10
patience = 100

class_weights = computing_class_weights(bees, 'subspecies')


In [ ]:
class_weights

In [ ]:
training_resnet152V2, model = utils.train(model,
                train_X,
                train_y,
                batch_size = batch_size,
                epochs = epochs,
                validation_data_X = val_X,
                validation_data_y = val_y,
                steps_per_epoch = steps_per_epoch,
                rotation_range = rotation_range,
                zoom_range = zoom_range,
                width_shift_range = width_shift_range,
                height_shift_range = height_shift_range,
                horizontal_flip = horizontal_flip,
                vertical_flip = vertical_flip,
                patience = patience,
                class_weights = class_weights
                               )

In [ ]:
utils.eval_model(training_resnet152V2, model, test_X, test_y, 'subspecies')

In [ ]:
df_subspecies = utils.load_test_and_generate_prediction_file(model, img_width, img_height, img_channels)
df_subspecies

# Combinando con la metadata
Combinando la información de las imágenes con la metadata disponible

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Assuming your dataframe is named df

# 1. Drop Unwanted Columns
df = df.drop(columns=['id', 'datetime', 'file', 'location', 'health', 'caste'])

# 2. Process `zip code`
encoder = OneHotEncoder(sparse=False)
zip_encoded = encoder.fit_transform(df[['zip code']])
zip_df = pd.DataFrame(zip_encoded, columns=encoder.get_feature_names(['zip_code']))
df = pd.concat([df, zip_df], axis=1).drop(columns=['zip code'])

# 3. Process `pollen_carrying`
df['pollen_carrying'] = df['pollen_carrying'].astype(int)


In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten, Embedding, concatenate
from tensorflow.keras.models import Model

image_model =
# Metadata input - assuming you've preprocessed it into a fixed size vector
metadata_input = Input(shape=(metadata_size,))
metadata_model = Dense(32, activation='relu')(metadata_input)

# Combine the features from both models
combined = concatenate([image_model, metadata_model])

combined_model = Dense(64, activation='relu')(combined)
output = Dense(num_classes, activation='softmax')(combined_model)

model = Model(inputs=[image_input, metadata_input], outputs=output)
